In [ ]:
import gym
import numpy as np
import copy, random, os, subprocess
from tensorflow.keras import layers, models

In [ ]:
MODEL_NAME = 'model04'
BUCKET = 'gs://etsuji-car-racing-v2'
os.environ['BUCKET'] = BUCKET
!gsutil mb -c regional -l us-west1 $BUCKET
!gsutil ls $BUCKET

In [ ]:
# Base model
class QValue:
    def __init__(self):
        self.model = self.build_model()

    def build_model(self):
        cnn_input = layers.Input(shape=(96, 96, 6), name='cnn_input')
        cnn1 = layers.Conv2D(32, (5, 5), padding='same',
                         use_bias=True, activation='relu',
                        name='cnn1')(cnn_input)
        pool1 = layers.MaxPooling2D((2, 2), name='pool1')(cnn1)
        cnn2 = layers.Conv2D(64, (5, 5), padding='same',
                         use_bias=True, activation='relu',
                        name='cnn2')(pool1)
        pool2 = layers.MaxPooling2D((2, 2), name='pool2')(cnn2)

        cnn_flatten = layers.Flatten(name='flatten')(pool2)
        action_input = layers.Input(shape=(5,), name='action_input')
        combined = layers.concatenate([cnn_flatten, action_input], name='concat')
        hidden1 = layers.Dense(1024, activation='relu', name='dense1')(combined)
        hidden2 = layers.Dense(512, activation='relu', name='dense2')(hidden1)
        q_value = layers.Dense(1, name='output')(hidden2)

        model = models.Model(inputs=[cnn_input, action_input], outputs=q_value)
        model.compile(loss='mse')
        return model

    def get_action(self, state):
        states = []
        actions = []
        for a in range(5):
            states.append(np.array(state))
            action_onehot = np.zeros(5)
            action_onehot[a] = 1
            actions.append(action_onehot)
  
        q_values = self.model.predict([np.array(states), np.array(actions)])
        optimal_action = np.argmax(q_values)
        return optimal_action, q_values[optimal_action][0]

In [ ]:
def join_frames(o0, o1):
    return np.r_[o0.transpose(), o1.transpose()].transpose() 

In [ ]:
def get_episode(environ, q_value, epsilon):
    episode = []
    o0 = environ.reset()
    o1 = copy.deepcopy(o0)
    total_r = 0

    if epsilon > 0:
        keep_count = 3
    else:
        keep_count = 1

    c = 0
    while True:
        if c % keep_count == 0: # Get new action
            if np.random.random() < epsilon:
                a = np.random.randint(5)
            else:
                a, _ = q_value.get_action(join_frames(o0, o1))
        c += 1
        o_new, r, done, inf = environ.step(a)                
        total_r += r

        # Terminate episode when total reward becomes negative
        if total_r < 0:
            done = 1

        if done:
            # Terminal state is to achive more than 990 or get out of the field.
            if total_r > 990 or r < -99:
                episode.append((join_frames(o0, o1), a, r, None))
            break
        else:
            episode.append((join_frames(o0, o1), a, r, join_frames(o1, o_new)))
        o0, o1 = o1, o_new

    print('epsilon={}, episode length={}, total rewards={}'.format(epsilon, len(episode), total_r))
    return episode, total_r

In [ ]:
def train(environ, q_value, epsilon, checkpoint=0):
    if checkpoint > 0:
        filename = 'car-racing-v2-{}-{}.hd5'.format(checkpoint, MODEL_NAME)
        subprocess.run(['gsutil', 'cp', '{}/{}/{}'.format(BUCKET, MODEL_NAME, filename), './'])
        print('load model {}'.format(filename))
        q_value.model = models.load_model(filename)
        os.remove(filename)

    experience = []
    good_experience = []
    best_r = [-100, -100, -100]

    for n in range(checkpoint + 1, checkpoint + 1000):
        print('iteration {}'.format(n))

        total_len = 0
        if n % 3 == 0:
            print('Testing the current performance...')
            episode, total_r = get_episode(environ, q_value, epsilon=0)
            with open('result.txt', 'a') as f:
                f.write('{},{},{},{}\n'.format(n, epsilon, len(episode), total_r))
            filename = 'car-racing-v2-{}-{}.hd5'.format(n, MODEL_NAME)
            q_value.model.save(filename, save_format='h5')
            subprocess.run(['gsutil', '-m', 'cp',
                            '{}'.format(filename), '{}/{}/'.format(BUCKET, MODEL_NAME)])
            os.remove(filename)
            experience += episode
            total_len += len(episode)

        while total_len < 500:
            episode, total_r = get_episode(environ, q_value, epsilon)
            total_len += len(episode)
            experience += episode

            # Keep the top 3 episodes
            if total_r > min(best_r):
                best_r = best_r[1:] + [total_r]
                good_experience += episode
                if len(good_experience) > 999 * 3:
                    good_experience = good_experience[-333 * 3:]

            
        if len(experience) > 999 * 5: # remember last 5 episodes
            experience = experience[-999 * 5:]

        epsilon = (epsilon - 0.2) * 0.99 + 0.2

        print('Training the model...')
        # Use latest episode + past episodes (sampling) + top 3 episode (sampling)
        latest_experience = experience[-total_len:]
        past_experience = experience[:-total_len]
        examples = latest_experience + \
            random.sample(past_experience, min(len(past_experience), 999)) + \
            random.sample(good_experience, min(len(good_experience), 999))
        
        # Show some statistics
        print('experience length={}'.format(len(experience)))
        print('number of examples={}'.format(len(examples)))
        print('best total reward = ', best_r)
        np.random.shuffle(examples)
                        
        states, actions, labels = [], [], []
        for state, a, r, state_new in examples:
            states.append(np.array(state))

            action_onehot = np.zeros(5)
            action_onehot[a] = 1
            actions.append(action_onehot)
            
            if state_new is None:   # Terminal state
                q_new = 0
            else:
                _, q_new = q_value.get_action(state_new)
            labels.append(np.array(r + q_new))

        hist = q_value.model.fit(
            [np.array(states), np.array(actions)], np.array(labels),
            batch_size=50, epochs=10, verbose=0)
        print('loss = {}'.format(hist.history['loss']))

In [ ]:
env = gym.make("CarRacing-v2", continuous=False)
q_value = QValue()
q_value.model.summary()

In [ ]:
train(env, q_value, epsilon=1.0, checkpoint=0)

loss = [2.768162488937378, 2.418365240097046, 2.3794500827789307, 1.9614886045455933, 2.720271587371826, 2.1824376583099365, 1.8873045444488525, 1.9147422313690186, 1.8142249584197998, 1.8815449476242065]
iteration 240
Testing the current performance...
epsilon=0, episode length=999, total rewards=54.16666666667104


Copying file://car-racing-v2-240-model04.hd5 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1/1 files][292.5 MiB/292.5 MiB] 100% Done                                    
Operation completed over 1 objects/292.5 MiB.                                    


Training the model...
experience length=4995
number of examples=2997
best total reward =  [62.1621621621666, 99.24812030075603, 125.19083969465893]
loss = [6.136407375335693, 3.137556552886963, 2.5942118167877197, 2.057283401489258, 2.742255210876465, 1.829239010810852, 2.256437063217163, 2.2141432762145996, 2.049298048019409, 2.480623483657837]
iteration 241
epsilon=0.2717028949618601, episode length=805, total rewards=-0.013919413919859042
Training the model...
experience length=4995
number of examples=2803
best total reward =  [62.1621621621666, 99.24812030075603, 125.19083969465893]
loss = [3.166813850402832, 2.1915152072906494, 2.469918966293335, 2.0943491458892822, 1.9838260412216187, 1.8812265396118164, 1.8309210538864136, 1.8809926509857178, 1.7958029508590698, 1.6844624280929565]
iteration 242
epsilon=0.2709858660122415, episode length=421, total rewards=-0.031325301205059136
epsilon=0.2709858660122415, episode length=559, total rewards=-0.0993788819880177
Training the model..

Copying file://car-racing-v2-243-model04.hd5 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1/1 files][292.5 MiB/292.5 MiB] 100% Done                                    
Operation completed over 1 objects/292.5 MiB.                                    


Training the model...
experience length=4995
number of examples=2636
best total reward =  [62.1621621621666, 99.24812030075603, 125.19083969465893]
loss = [3.7625041007995605, 2.9136412143707275, 2.0801773071289062, 2.2903549671173096, 1.8444706201553345, 1.7778537273406982, 2.1314308643341064, 1.7004762887954712, 2.7045981884002686, 1.5179171562194824]
iteration 244
epsilon=0.2695732472785979, episode length=687, total rewards=-0.07147766323078142
Training the model...
experience length=4995
number of examples=2685
best total reward =  [62.1621621621666, 99.24812030075603, 125.19083969465893]
loss = [3.672888994216919, 1.9005690813064575, 1.8988759517669678, 1.7187613248825073, 1.7307686805725098, 1.6953930854797363, 1.7075167894363403, 3.230276346206665, 1.64467453956604, 1.8094605207443237]
iteration 245
epsilon=0.2688775148058119, episode length=718, total rewards=-0.004575163399338694
Training the model...
experience length=4995
number of examples=2716
best total reward =  [62.162

...

Training the model...
experience length=4995
number of examples=2997
best total reward =  [495.6678700360936, 517.328519855586, 561.9217081850394]
loss = [17.662343978881836, 13.679000854492188, 11.480753898620605, 9.772933959960938, 9.17158317565918, 7.909023761749268, 7.399991035461426, 7.173945426940918, 7.047061920166016, 6.503732204437256]
iteration 511
epsilon=0.20475378167016733, episode length=999, total rewards=160.86956521739455
Training the model...
experience length=4995
number of examples=2997
best total reward =  [495.6678700360936, 517.328519855586, 561.9217081850394]
loss = [12.005399703979492, 10.276866912841797, 8.876493453979492, 8.15312671661377, 7.276163101196289, 6.944616317749023, 6.721719264984131, 6.467621803283691, 5.766514778137207, 6.414693355560303]
iteration 512
epsilon=0.20470624385346567, episode length=999, total rewards=331.0954063604138
Training the model...
experience length=4995
number of examples=2997
best total reward =  [495.6678700360936, 517.32

Copying file://car-racing-v2-513-model04.hd5 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1/1 files][292.5 MiB/292.5 MiB] 100% Done                                    
Operation completed over 1 objects/292.5 MiB.                                    


Training the model...
experience length=4995
number of examples=2870
best total reward =  [495.6678700360936, 517.328519855586, 561.9217081850394]
loss = [42.63509750366211, 30.229385375976562, 27.76258659362793, 18.120588302612305, 14.273712158203125, 17.710819244384766, 9.443156242370605, 9.106833457946777, 8.63056468963623, 11.16848087310791]
iteration 514
epsilon=0.20461258960078169, episode length=999, total rewards=269.49152542371877
Training the model...
experience length=4995
number of examples=2997
best total reward =  [495.6678700360936, 517.328519855586, 561.9217081850394]
loss = [17.230581283569336, 12.57273006439209, 11.523417472839355, 9.262636184692383, 10.253412246704102, 8.011290550231934, 7.721951961517334, 6.618598937988281, 7.128584861755371, 7.605776786804199]
iteration 515
epsilon=0.20456646370477388, episode length=999, total rewards=593.1407942238181
Training the model...
experience length=4995
number of examples=2997
best total reward =  [517.328519855586, 561.

Copying file://car-racing-v2-516-model04.hd5 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1/1 files][292.5 MiB/292.5 MiB] 100% Done                                    
Operation completed over 1 objects/292.5 MiB.                                    


Training the model...
experience length=4995
number of examples=2997
best total reward =  [517.328519855586, 561.9217081850394, 593.1407942238181]
loss = [17.740522384643555, 12.548669815063477, 11.698053359985352, 9.494993209838867, 9.09181022644043, 12.161459922790527, 6.980846881866455, 8.417120933532715, 6.939248561859131, 6.674428462982178]
iteration 517
epsilon=0.20447559107704888, episode length=999, total rewards=372.6027397260217
Training the model...
experience length=4995
number of examples=2997
best total reward =  [517.328519855586, 561.9217081850394, 593.1407942238181]
